# Environment Setup for AG News Text Classification

## Overview

This tutorial guides through complete environment setup following best practices from:
- Tatman et al. (2018): "A Practical Guide to Training Restricted Boltzmann Machines"
- Sculley et al. (2015): "Hidden Technical Debt in Machine Learning Systems"

### Tutorial Objectives
1. Verify Python environment and dependencies
2. Configure GPU support for deep learning
3. Setup data directories and paths
4. Validate project installation
5. Configure logging and monitoring

Author: Võ Hải Dũng  
Email: vohaidung.work@gmail.com  
Date: 2025

## 1. Python Environment Verification

In [ ]:
# Standard library imports
import sys
import os
import platform
import subprocess
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import warnings

# Check Python version
print("System Information")
print("="*60)
print(f"Operating System: {platform.system()} {platform.release()}")
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")
print(f"Python Version: {sys.version}")
print(f"Python Executable: {sys.executable}")

# Verify Python version meets requirements
required_version = (3, 8)
current_version = sys.version_info[:2]

if current_version < required_version:
    raise SystemError(f"Python {required_version[0]}.{required_version[1]}+ required. "
                     f"Current: {current_version[0]}.{current_version[1]}")
else:
    print(f"\n✓ Python version check passed: {current_version[0]}.{current_version[1]}")

## 2. Core Dependencies Installation Check

In [ ]:
def check_package_installation(packages: Dict[str, str]) -> Dict[str, Dict[str, str]]:
    """
    Check installation status of required packages.
    
    Following dependency management practices from:
    - McMahan & Streeter (2014): "Delay-Tolerant Algorithms for Asynchronous Distributed Online Learning"
    """
    results = {}
    
    for package_name, import_name in packages.items():
        try:
            if import_name:
                module = __import__(import_name)
                version = getattr(module, '__version__', 'unknown')
            else:
                # For packages without direct import
                import importlib.metadata
                version = importlib.metadata.version(package_name)
            
            results[package_name] = {
                'status': 'installed',
                'version': version
            }
        except (ImportError, importlib.metadata.PackageNotFoundError):
            results[package_name] = {
                'status': 'missing',
                'version': None
            }
    
    return results

# Define core packages
core_packages = {
    'numpy': 'numpy',
    'pandas': 'pandas',
    'scikit-learn': 'sklearn',
    'torch': 'torch',
    'transformers': 'transformers',
    'datasets': 'datasets',
    'tokenizers': 'tokenizers',
    'accelerate': 'accelerate',
    'peft': 'peft',
    'fastapi': 'fastapi',
    'uvicorn': 'uvicorn',
    'grpcio': 'grpc',
    'protobuf': 'google.protobuf',
    'pydantic': 'pydantic'
}

print("\nCore Package Installation Status")
print("="*60)

installation_status = check_package_installation(core_packages)
missing_packages = []

for package, info in installation_status.items():
    if info['status'] == 'installed':
        print(f"✓ {package:20} : {info['version']}")
    else:
        print(f"✗ {package:20} : NOT INSTALLED")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠ Missing packages: {', '.join(missing_packages)}")
    print(f"Install with: pip install {' '.join(missing_packages)}")
else:
    print("\n✓ All core packages installed successfully")

## 3. GPU Configuration and CUDA Setup

In [ ]:
def check_gpu_availability() -> Dict[str, any]:
    """
    Check GPU availability and CUDA configuration.
    
    Following GPU setup guidelines from:
    - Jouppi et al. (2017): "In-Datacenter Performance Analysis of a Tensor Processing Unit"
    """
    gpu_info = {
        'available': False,
        'cuda_version': None,
        'device_count': 0,
        'devices': []
    }
    
    try:
        import torch
        
        gpu_info['available'] = torch.cuda.is_available()
        
        if gpu_info['available']:
            gpu_info['cuda_version'] = torch.version.cuda
            gpu_info['device_count'] = torch.cuda.device_count()
            
            for i in range(gpu_info['device_count']):
                device_props = torch.cuda.get_device_properties(i)
                gpu_info['devices'].append({
                    'index': i,
                    'name': device_props.name,
                    'memory': f"{device_props.total_memory / 1e9:.2f} GB",
                    'capability': f"{device_props.major}.{device_props.minor}"
                })
    except ImportError:
        print("PyTorch not installed. Cannot check GPU availability.")
    
    return gpu_info

print("\nGPU Configuration")
print("="*60)

gpu_info = check_gpu_availability()

if gpu_info['available']:
    print(f"✓ CUDA Available: {gpu_info['cuda_version']}")
    print(f"✓ GPU Count: {gpu_info['device_count']}")
    
    for device in gpu_info['devices']:
        print(f"\nGPU {device['index']}:")
        print(f"  Name: {device['name']}")
        print(f"  Memory: {device['memory']}")
        print(f"  Compute Capability: {device['capability']}")
else:
    print("✗ No GPU available. Training will use CPU.")
    print("\nFor GPU support:")
    print("  1. Install CUDA Toolkit")
    print("  2. Install PyTorch with CUDA support")
    print("  3. Verify GPU drivers are installed")

# Set device for future use
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nDefault device set to: {device}")

## 4. Project Structure Setup

In [ ]:
# Setup project paths
PROJECT_ROOT = Path("../..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

print("Project Structure Verification")
print("="*60)
print(f"Project Root: {PROJECT_ROOT}")

# Define required directories
required_dirs = [
    'data/raw',
    'data/processed',
    'data/augmented',
    'data/external',
    'data/cache',
    'outputs/models',
    'outputs/results',
    'outputs/logs',
    'outputs/analysis',
    'configs',
    'src'
]

# Check and create directories
print("\nDirectory Structure:")
for dir_path in required_dirs:
    full_path = PROJECT_ROOT / dir_path
    if full_path.exists():
        print(f"✓ {dir_path}")
    else:
        full_path.mkdir(parents=True, exist_ok=True)
        print(f"✓ {dir_path} (created)")

# Verify critical files
critical_files = [
    'setup.py',
    'requirements/base.txt',
    'configs/constants.py',
    'src/__init__.py'
]

print("\nCritical Files:")
missing_files = []
for file_path in critical_files:
    full_path = PROJECT_ROOT / file_path
    if full_path.exists():
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} (missing)")
        missing_files.append(file_path)

if missing_files:
    print(f"\n⚠ Missing critical files: {', '.join(missing_files)}")
else:
    print("\n✓ All critical files present")

## 5. Import Project Modules

In [ ]:
print("Testing Project Module Imports")
print("="*60)

# Test critical imports
test_imports = [
    ('Core Registry', 'src.core.registry'),
    ('Core Factory', 'src.core.factory'),
    ('Data Module', 'src.data'),
    ('Models Module', 'src.models'),
    ('Training Module', 'src.training'),
    ('Evaluation Module', 'src.evaluation'),
    ('API Module', 'src.api'),
    ('Services Module', 'src.services'),
    ('Utils Module', 'src.utils'),
    ('Config Loader', 'configs.config_loader')
]

import_errors = []

for module_name, import_path in test_imports:
    try:
        module = __import__(import_path, fromlist=[''])
        print(f"✓ {module_name:20} : {import_path}")
    except ImportError as e:
        print(f"✗ {module_name:20} : {str(e)}")
        import_errors.append((module_name, str(e)))

if import_errors:
    print("\n⚠ Import Errors Detected:")
    for name, error in import_errors:
        print(f"  {name}: {error}")
else:
    print("\n✓ All project modules imported successfully")

## 6. Configuration Validation

In [ ]:
# Load and validate configurations
from configs.constants import (
    AG_NEWS_CLASSES,
    AG_NEWS_NUM_CLASSES,
    LABEL_TO_ID,
    ID_TO_LABEL,
    DATA_DIR,
    MODEL_DIR,
    LOG_DIR
)
from configs.config_loader import ConfigLoader

print("Configuration Validation")
print("="*60)

# Validate constants
print("AG News Configuration:")
print(f"  Classes: {AG_NEWS_CLASSES}")
print(f"  Number of Classes: {AG_NEWS_NUM_CLASSES}")
print(f"  Label Mapping: {LABEL_TO_ID}")

# Validate paths
print("\nPath Configuration:")
print(f"  Data Directory: {DATA_DIR}")
print(f"  Model Directory: {MODEL_DIR}")
print(f"  Log Directory: {LOG_DIR}")

# Test configuration loading
try:
    config_loader = ConfigLoader()
    
    # Test loading different config types
    test_configs = [
        'environments/dev.yaml',
        'models/single/deberta_v3_xlarge.yaml',
        'training/standard/base_training.yaml'
    ]
    
    print("\nTesting Configuration Files:")
    for config_path in test_configs:
        try:
            config = config_loader.load_config(config_path)
            print(f"✓ {config_path}")
        except Exception as e:
            print(f"✗ {config_path}: {str(e)}")
            
except Exception as e:
    print(f"\n⚠ Configuration loading error: {str(e)}")

## 7. Memory and Resource Check

In [ ]:
import psutil

def get_system_resources() -> Dict[str, any]:
    """
    Get system resource information.
    
    Following resource monitoring practices from:
    - Dean et al. (2012): "Large Scale Distributed Deep Networks"
    """
    return {
        'cpu': {
            'count': psutil.cpu_count(logical=False),
            'count_logical': psutil.cpu_count(logical=True),
            'percent': psutil.cpu_percent(interval=1),
            'freq': psutil.cpu_freq().current if psutil.cpu_freq() else None
        },
        'memory': {
            'total': psutil.virtual_memory().total / (1024**3),
            'available': psutil.virtual_memory().available / (1024**3),
            'percent': psutil.virtual_memory().percent
        },
        'disk': {
            'total': psutil.disk_usage('/').total / (1024**3),
            'free': psutil.disk_usage('/').free / (1024**3),
            'percent': psutil.disk_usage('/').percent
        }
    }

print("System Resources")
print("="*60)

resources = get_system_resources()

print("CPU:")
print(f"  Physical Cores: {resources['cpu']['count']}")
print(f"  Logical Cores: {resources['cpu']['count_logical']}")
print(f"  Current Usage: {resources['cpu']['percent']:.1f}%")
if resources['cpu']['freq']:
    print(f"  Frequency: {resources['cpu']['freq']:.0f} MHz")

print("\nMemory:")
print(f"  Total: {resources['memory']['total']:.2f} GB")
print(f"  Available: {resources['memory']['available']:.2f} GB")
print(f"  Usage: {resources['memory']['percent']:.1f}%")

print("\nDisk:")
print(f"  Total: {resources['disk']['total']:.2f} GB")
print(f"  Free: {resources['disk']['free']:.2f} GB")
print(f"  Usage: {resources['disk']['percent']:.1f}%")

# Check if resources are sufficient
print("\nResource Adequacy Check:")
min_memory = 8  # GB
min_disk = 10  # GB

if resources['memory']['available'] >= min_memory:
    print(f"✓ Memory: Sufficient ({resources['memory']['available']:.1f} GB available)")
else:
    print(f"⚠ Memory: May be insufficient ({resources['memory']['available']:.1f} GB < {min_memory} GB)")

if resources['disk']['free'] >= min_disk:
    print(f"✓ Disk: Sufficient ({resources['disk']['free']:.1f} GB free)")
else:
    print(f"⚠ Disk: May be insufficient ({resources['disk']['free']:.1f} GB < {min_disk} GB)")

## 8. Network and API Connectivity

In [ ]:
import requests
import socket

print("Network Connectivity Check")
print("="*60)

# Check internet connectivity
def check_internet_connection() -> bool:
    """Check basic internet connectivity."""
    try:
        socket.create_connection(("8.8.8.8", 53), timeout=3)
        return True
    except OSError:
        return False

if check_internet_connection():
    print("✓ Internet connection available")
else:
    print("✗ No internet connection")

# Check access to key services
services_to_check = [
    ('Hugging Face Hub', 'https://huggingface.co'),
    ('PyTorch Hub', 'https://pytorch.org'),
    ('GitHub', 'https://github.com'),
    ('Google Colab', 'https://colab.research.google.com')
]

print("\nService Accessibility:")
for service_name, url in services_to_check:
    try:
        response = requests.head(url, timeout=5)
        if response.status_code < 400:
            print(f"✓ {service_name:20} : Accessible")
        else:
            print(f"⚠ {service_name:20} : Status {response.status_code}")
    except requests.RequestException as e:
        print(f"✗ {service_name:20} : Not accessible")

# Check Hugging Face model access
print("\nHugging Face Model Access:")
test_models = [
    'microsoft/deberta-v3-base',
    'roberta-base',
    'google/electra-base-discriminator'
]

for model_name in test_models:
    url = f"https://huggingface.co/{model_name}"
    try:
        response = requests.head(url, timeout=5)
        if response.status_code < 400:
            print(f"✓ {model_name}")
        else:
            print(f"⚠ {model_name} : Status {response.status_code}")
    except:
        print(f"✗ {model_name} : Not accessible")

## 9. Environment Summary and Recommendations

In [ ]:
print("Environment Setup Summary")
print("="*60)

# Compile status
setup_status = {
    'python': current_version >= required_version,
    'packages': len(missing_packages) == 0,
    'gpu': gpu_info['available'],
    'project_structure': len(missing_files) == 0,
    'imports': len(import_errors) == 0,
    'memory': resources['memory']['available'] >= min_memory,
    'disk': resources['disk']['free'] >= min_disk,
    'internet': check_internet_connection()
}

# Overall status
all_ready = all(setup_status.values())
critical_ready = setup_status['python'] and setup_status['packages'] and setup_status['project_structure']

print("Status Overview:")
for component, status in setup_status.items():
    icon = "✓" if status else "✗"
    print(f"{icon} {component.replace('_', ' ').title():20} : {'Ready' if status else 'Issues Detected'}")

print("\n" + "="*60)
if all_ready:
    print("✓ ENVIRONMENT FULLY CONFIGURED")
    print("\nYou are ready to proceed with:")
    print("  1. Data loading and preprocessing")
    print("  2. Model training and evaluation")
    print("  3. API deployment and testing")
elif critical_ready:
    print("⚠ ENVIRONMENT PARTIALLY CONFIGURED")
    print("\nCritical components are ready, but some optimizations missing:")
    if not setup_status['gpu']:
        print("  - GPU not available (training will be slower)")
    if not setup_status['memory']:
        print("  - Limited memory (may affect batch sizes)")
else:
    print("✗ ENVIRONMENT SETUP INCOMPLETE")
    print("\nPlease resolve critical issues before proceeding.")

# Save environment report
from src.utils.io_utils import safe_save, ensure_dir

output_dir = PROJECT_ROOT / "outputs" / "setup"
ensure_dir(output_dir)

environment_report = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'system': {
        'platform': platform.platform(),
        'python_version': f"{current_version[0]}.{current_version[1]}",
        'cuda_available': gpu_info['available'],
        'gpu_count': gpu_info['device_count']
    },
    'resources': resources,
    'status': setup_status,
    'ready': all_ready
}

report_path = output_dir / "environment_report.json"
safe_save(environment_report, report_path)
print(f"\nEnvironment report saved to: {report_path}")

## 10. Next Steps

### Recommended Tutorial Progression

Based on your environment setup, proceed with the following tutorials:

1. **01_data_loading_basics.ipynb**
   - Learn to load and explore AG News dataset
   - Understand data structures and formats
   - Practice basic data operations

2. **02_preprocessing_tutorial.ipynb**
   - Apply text cleaning techniques
   - Implement tokenization strategies
   - Create feature representations

3. **03_model_training_basics.ipynb**
   - Train baseline models
   - Understand training loops
   - Monitor training progress

4. **04_evaluation_tutorial.ipynb**
   - Evaluate model performance
   - Generate comprehensive reports
   - Compare different models

### Troubleshooting Resources

If you encounter issues:
- Check `TROUBLESHOOTING.md` for common problems
- Review logs in `outputs/logs/`
- Consult project documentation in `docs/`
- Submit issues to project repository

### Performance Tips

For optimal performance:
- Use GPU acceleration when available
- Enable mixed precision training
- Implement gradient accumulation for large models
- Utilize caching for preprocessed data
- Monitor memory usage during training